# 1. 字符问题

“字符串”是个相当简单的概念：一个字符串是一个字符序列。问题出在“字符”的定义上。  
在 2015 年， “字符”的最佳定义是 Unicode 字符。从 Python 3 的 str 对象中获取的
元素是 Unicode 字符，这相当于从 Python 2 的 unicode 对象中获取的元素，而不是从
Python 2 的 str 对象中获取的原始字节序列。

Unicode 标准把字符的标识和具体的字节表述进行了如下的明确区分。
- 字符的标识，即码位。例如，字母 A 的码位是 U+0041，
- 字符的具体表述取决于所用的编码。编码是在码位和字节序列之间转换时使用的算法。在 UTF-8 编码中， A（ U+0041）的码位编码成单个字节 \x41，而在UTF-16LE编码中编码成两个字节 \x41\x00。

In [10]:
s = 'café'

In [11]:
len(s)

4

In [13]:
b = s.encode('utf-8')
print(b)
print(len(b))

b'caf\xc3\xa9'
5


In [14]:
b.decode('utf-8')

'café'

# 2. 字节概要

bytes 或 bytearray 对象的各个元素是介于 0~255（含）之间的整数，而不像 Python 2
的 str 对象那样是单个的字符。然而，二进制序列的切片始终是同一类型的二进制序
列，包括长度为 1 的切片，

In [15]:
bs = bytes('café', encoding='utf_8')

In [16]:
bs

b'caf\xc3\xa9'

In [18]:
bs[-1]

169

In [19]:
bs[:1]

b'c'

In [21]:
bs_arr = bytearray(bs)
bs_arr

bytearray(b'caf\xc3\xa9')

In [23]:
bs_arr[:1]

bytearray(b'c')

如果正则表达式编译自二进制序列而不是字符串， re 模块中的正则表达式函数也能处理二进制序列。

二进制序列有个类方法是 str 没有的，名为 fromhex，它的作用是解析十六进制数字对
（数字对之间的空格是可选的），构建二进制序列：

In [29]:
bs_num = bytes.fromhex('000102')
bs_num

b'\x00\x01\x02'

构建 bytes 或 bytearray 实例还可以调用各自的构造方法，传入下述参数。
- 一个 str 对象和一个 encoding 关键字参数。
- 一个可迭代对象，提供 0~255 之间的数值。
- 一个实现了缓冲协议的对象（如bytes、 bytearray、 memoryview、 array.array）；此时，把源对象中的字节序列复制到新建的二进制序列中。

# 3. 基本的编解码器

Python 自带了超过 100 种编解码器（ codec, encoder/decoder），用于在文本和字节之间相互转换。

In [37]:
for codec in ['latin_1', 'utf_8', 'utf_16']:
    print(codec, 'El Niño'.encode(codec), sep='\t')

latin_1	b'El Ni\xf1o'
utf_8	b'El Ni\xc3\xb1o'
utf_16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


In [35]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



# 4. 了解编解码问题

出现与 Unicode 有关的错误时，首先要明确异常的类型。导致编码问题的是UnicodeEncodeError、 UnicodeDecodeError，还是如 SyntaxError 的其他错误？解决问题之前必须清楚这一点。

- 多数非 UTF 编解码器只能处理 Unicode 字符的一小部分子集。把文本转换成字节序列时，如果目标编码中没有定义某个字符，那就会抛出 UnicodeEncodeError 异常，除非把 errors 参数传给编码方法或函数，对错误进行特殊处理。

In [38]:
city = 'São Paulo'

In [39]:
city.encode('utf-8')

b'S\xc3\xa3o Paulo'

In [40]:
city.encode('utf-16')

b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'

In [41]:
city.encode('cp437')

UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 1: character maps to <undefined>

In [42]:
city.encode('cp437', errors='ignore')

b'So Paulo'

In [43]:
city.encode('cp437', errors='replace')

b'S?o Paulo'

In [45]:
city.encode('cp437', errors='xmlcharrefreplace') 

b'S&#227;o Paulo'

编解码器的错误处理方式是可扩展的。你可以为 errors 参数注册额外的字符串，方法是把一个名称和一个错误处理函数传给 codecs.register_error 函数。
参见 codecs.register_error 函数的文档

- 不是每一个字节都包含有效的 ASCII 字符，也不是每一个字符序列都是有效的 UTF-8 或UTF-16。因此，把二进制序列转换成文本时，如果假设是这两个编码中的一个，遇到无法转换的字节序列时会抛出 UnicodeDecodeError。

In [46]:
octets = b'Montr\xe9al'

In [47]:
octets.decode('cp1252')

'Montréal'

In [48]:
octets.decode('iso8859_7') 

'Montrιal'

In [49]:
octets.decode('utf_8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [50]:
octets.decode('utf_8', errors='replace')

'Montr�al'

- 使用预期之外的编码加载模块时抛出的SyntaxError

Python 3 默认使用 UTF-8 编码源码， Python 2（从 2.5 开始）则默认使用 ASCII。如果加载
的 .py 模块中包含 UTF-8 之外的数据，而且没有声明编码，会抛出错误

# 5. 处理文本文件

处理文本的最佳实践是“Unicode 三明治”：  
- 要尽早把输入（例如读取文件时）的字节序列解码成字符串
- 业务逻辑，只能处理字符串对象
- 对输出来说，则要尽量晚地把字符串编码成字节序列。